# Fetch List of Cases

In [23]:
import requests
import pandas as pd
import numpy as np
import re
import pickle

from bs4 import BeautifulSoup
from selenium import webdriver
from tqdm import tqdm_notebook as tqdm

In [2]:
data = pd.DataFrame()

In [4]:
for i in tqdm(range(49,0,-1)):
    data_page = []
    # set the URL
    url = 'http://sipp.pn-jakartapusat.go.id/http://sipp.pn-jakartapusat.go.id/list_perkara/page/'+str(i)+'/OXB2Nkc3U0lzUVoza1pGcjMrNUx6QkZyRU5ZU1cxekczRnYrcGY0ZDN3elVpKzFjeDJjWXAzOWRTOU9ENy81ejFFUlFSRzkxWURhaWVzNWVKYUdYTGc9PQ==/key/col/2:147'
    
    # fetch the webpage
    driver = webdriver.Chrome()
    driver.implicitly_wait(20)
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    
    # parsing data row by row
    table_rows = soup.find_all('tr')
    for row in table_rows:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') if ele.get_text() != '[detil]' else ele.find('a')['href']  for ele in cols]
        data_page.append(cols)
    
    page_table = pd.DataFrame(data_page[1:-1],columns=data_page[0]).set_index('Nomor Perkara').drop('No',axis=1)
    data = data.append(page_table)

A Jupyter Widget

In [12]:
# Remove duplicates by case ID
data = data[~data.index.duplicated(keep='first')]

In [26]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'wb') as f:
    pickle.dump(data, f)

# Crawled data to CSV
data.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.csv')

# Fetch Case Details

## Fetch Judge Name

In [116]:
# Load pickled case list
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_list.pickle', 'rb') as f:
    datax = pickle.load(f)

In [117]:
case_list = list(datax.index.unique())
len(case_list)

964

In [118]:
data_hakim = pd.DataFrame()

In [119]:
for case in tqdm(case_list):
    url = datax.loc[case]['Link']
    url_hakim = url.replace("show_detil","show_penetapan")
    
    # fetch the webpage
    driver = webdriver.Chrome()
    driver.implicitly_wait(20)
    driver.get(url_hakim)
    soup = BeautifulSoup(driver.page_source, "lxml")
    driver.quit()
    
    # Tabel Hakim
    kolom_tabel_hakim = ['Nomor Perkara','Tanggal Penetapan', 'Nama Hakim/Majelis Hakim', 'Posisi', 'Aktif']
    hakim_kasus = []
    table_hakim = soup.find_all('div',class_='cssTable')[0].find_all('tr')
    for row in table_hakim:
        cols = row.find_all('td')
        cols = [ele.get_text(separator=' ') if ele.get_text() != '[detil]' else ele.find('a')['href']  for ele in cols]
        cols.insert(0,case)
        hakim_kasus.append(cols)
    
    page_table = pd.DataFrame(hakim_kasus[1:],columns=kolom_tabel_hakim).set_index('Nomor Perkara')
    data_hakim = data_hakim.append(page_table)

A Jupyter Widget

In [120]:
data_hakim

,Tanggal Penetapan,Nama Hakim/Majelis Hakim,Posisi,Aktif
Nomor Perkara,,,,
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Ketua,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
58/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
57/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Ketua,Ya
57/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
57/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya
57/Pid.Sus-TPK/2019/PN Jkt.Pst,"Senin, 29 Apr. 2019",Belum Dapat Ditampilkan,Hakim Anggota,Ya


In [122]:
# Save data to external file

# Crawled DataFrame to pickle
with open('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_judge_list.pickle', 'wb') as f:
    pickle.dump(data_hakim, f)

# Crawled data to CSV
data_hakim.to_csv('C:/Users/akmal.fadhlurrahman/Documents/personal_stuff/adhoc_judge/data/case_judge_list.csv')